In [1]:
import numpy as np
from sklearn.svm import SVR
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.cluster import KMeans
import keras
import tensorflow as tf
from keras.layers import Layer
from keras import backend as K
from keras.layers import Dense, Flatten
from keras.models import Sequential
from keras.losses import binary_crossentropy

import matplotlib.pyplot as plt
import pickle
import csv
import pandas as pd
import os
import directories
import natsort
import glob
import time

In [2]:
def rbf(x, c, s):
    return np.exp(-1 / (2 * s**2) * (x-c)**2)

In [22]:
class RBFNet(Layer):
    def __init__(self, units, gamma, **kwargs):
        super(RBFNet, self).__init__(**kwargs)
        self.units = units
        self.gamma = K.cast_to_floatx(gamma)

    def build(self, input_shape):
#         print(input_shape)
#         print(self.units)
        self.mu = self.add_weight(name='mu',
                                  shape=(int(input_shape[1]), self.units),
                                  initializer='uniform',
                                  trainable=True)
        super(RBFNet, self).build(input_shape)

    def call(self, inputs):
        diff = K.expand_dims(inputs) - self.mu
        l2 = K.sum(K.pow(diff, 2), axis=1)
        res = K.exp(-1 * self.gamma * l2)
        return res

    def compute_output_shape(self, input_shape):
        return (input_shape[0], self.units)

In [29]:
# sample inputs and add noise
NUM_SAMPLES = 100
X = np.random.uniform(0., 1., NUM_SAMPLES)
X = np.sort(X, axis=0)
X = X.reshape(-1, 1)
noise = np.random.uniform(-0.1, 0.1, NUM_SAMPLES)
y = np.sin(2 * np.pi * X)  + noise
# y = y.reshape(-1,100)
model = Sequential()
model.add(Flatten(input_shape=(1,1)))
model.add(RBFNet(10, 0.5))
model.add(Dense(1, activation='sigmoid', name='foo'))
model.compile(optimizer='rmsprop', loss=binary_crossentropy)
model.fit(X, y, batch_size=256, epochs=3)
y_pred = model.predict(X)
plt.plot(X, y, '-o', label='true')
plt.plot(X, y_pred, '-o', label='RBF-Net')
plt.legend()
plt.tight_layout()
plt.show()

IndexError: list index out of range

In [ ]:
def train_creep(p_collection_, r2_):
    print('start training creep')
    csv_i = []
    for i, csv_file in enumerate(natsort.natsorted(glob.glob(p_collection_ + "/input/creep_folder/*.csv"))):
        data = np.loadtxt(csv_file, delimiter=",")
        # data = np.delete(data, zero_nodes)
        data = data.reshape(1, len(data))
        data_i = data.flatten()
        csv_i.append(data_i)
        # print(data)
        data_out = np.array(csv_i)
        data_out = np.nan_to_num(data_out, copy=False, nan=0.1)
        data_out = data_out[:, 16237:16238].flatten()
    # print(f'y: {data_out}')
    print(f'len_data_out: {len(data_out)}')

    # input_ = pd.read_csv(p_collection + "/input/input.csv", header=None)
    input_ = pd.read_csv(p_collection_ + "/input/temperature_creep.csv", header=None)
    x = input_.iloc[:, :2].values
    # print(f'x: {x}')
    print(f'len_x: {len(x)}')

    r2_train = 0.0
    r2_test = 0.0
    max_ = 0
    
    while r2_test < float(r2_):
        
        X_train, X_test, Y_train, Y_test = train_test_split(x, data_out, test_size=0.33)

        svr_rbf = RBFNet(lr=1e-2, k=2)
        rbf_model = svr_rbf.fit(X_train, Y_train)

        y_train_predicted = rbf_model.predict(X_train)
        y_test_predicted = rbf_model.predict(X_test)

        # Evaluating training dataset
        rmse_train = np.sqrt(mean_squared_error(Y_train, y_train_predicted))
        r2_train = r2_score(Y_train, y_train_predicted)
        mae_train = mean_absolute_error(Y_train, y_train_predicted)

        # Evaluating test dataset
        rmse_test = np.sqrt(mean_squared_error(Y_test, y_test_predicted))
        r2_test = r2_score(Y_test, y_test_predicted)
        mae_test = mean_absolute_error(Y_test, y_test_predicted)
        
        if r2_test > max_:
            max_ = r2_test
            print("R2 score of test is {}".format(max_))
    
    print("-------------------------------------------")
    print("RMSE of training is {}".format(rmse_train))
    print("R2 score of training  is {}".format(r2_train))
    print("MAE of training is {}".format(mae_train))
    print("RMSE of test is {}".format(rmse_test))
    print("R2 score of test is {}".format(max_))
    print("MAE of test is {}".format(mae_test))
    print('Done training')
    
    pickle.dump(rbf_model, open(os.path.join(p_collection_, 'output/training_data_creep.pkl'), "wb"),pickle.HIGHEST_PROTOCOL)

In [ ]:
if __name__ == '__main__':
    p_collection_path = r"A:/Nguyen_Minh_Tuan/1_PROJECTS/2_QatarSMR/2.ML_test"
    stress_output = "A:/Nguyen_Minh_Tuan/1_PROJECTS/2_QatarSMR/2.ML_test/output/training_data_sig_11.pkl"
    output_folder = 'A:/Nguyen_Minh_Tuan/1_PROJECTS/2_QatarSMR/2.ML_test/output/'
    r2 = 0.95
    
    if not os.path.isfile(stress_output):
        start = time.time()
        # train_sig_11(p_collection_path, r2)
        # train_sig_22(p_collection_path, r2)
        # train_sig_33(p_collection_path, r2)
        # train_sig_12(p_collection_path, r2)
        # train_sig_13(p_collection_path, r2)
        # train_sig_23(p_collection_path, r2)
        train_creep(p_collection_path, r2)
        train_time = time.time()
        print(f'DONE Training\n Training time is: {train_time - start}')
        input_temp = 1174
        print(input_temp)
        input_time = 3
        print(input_time)
        start = time.time()
        # stress_prediction = predict_based_on_ml_model(output_folder, input_temp, input_time)
        # von_mises = calculate_von_mises_from_stress_tensor(stress_prediction)
        # predicted_creep = creep_prediction(output_folder, input_temp, input_time)
        # print(predicted_creep)
        cal_dmg = time.time()
        print(f'Time for Calculation of von Mises: {cal_dmg - start}')
    else:
        input_temp = 1174
        print(input_temp)
        input_time = 3
        print(input_time)
        start = time.time()
        # stress_prediction = predict_based_on_ml_model(output_folder, input_temp, input_time)
        # von_mises = calculate_von_mises_from_stress_tensor(stress_prediction)
        # print(von_mises)
        # predicted_creep = creep_prediction(output_folder, input_temp, input_time)
        # print(predicted_creep)
        cal_dmg = time.time()
        print(f'Time for Calculation of von Mises: {cal_dmg - start}')